# Imports

In [1]:
# Imports gerais
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.point import Point
from scipy import stats
import duckdb
from datetime import datetime

# Importar k-means e silhouete score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler

# Warnings
import warnings
warnings.filterwarnings("ignore")

# Excel
import openpyxl
from openpyxl.styles import Font, PatternFill
from openpyxl.styles.borders import Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows

# Testar algoritimo alerta

In [3]:
from unidecode import unidecode

def kb_data_cleaning(df):
    """
    Clean and preprocess the knowledge base dataframe.
    
    Args:
        df (pd.DataFrame): Input dataframe to clean
        
    Returns:
        pd.DataFrame: Cleaned dataframe
    """
    # Remove duplicates
    df = df.drop_duplicates(subset=["id"])
    
    # Filter bedrooms and bathrooms
    df = df.loc[(df["n_quartos"] > 0) & (df["n_quartos"] < 10)]
    df = df.loc[(df["n_banheiros"] > 0) & (df["n_banheiros"] < 10)]
    
    # Filter property size
    df = df.loc[(df["tamanho"] > 10) & (df["tamanho"] < 1000)]
    
    # Filter property price
    df = df.loc[(df["preco"] > 10_000) & (df["preco"] < 10_000_000)]
    
    # Remove null neighborhoods
    df = df.loc[df["bairro"].notna()]
    
    # Format neighborhood names
    df["bairro"] = df["bairro"].apply(lambda x: unidecode(x.lower().strip()))
    
    # Return dataframe
    return df

def leilao_data_cleaning(df):
    """
    Clean and preprocess the leilao dataframe.
    
    Args:
        df (pd.DataFrame): Input dataframe to clean
        
    Returns:
        pd.DataFrame: Cleaned dataframe
    """    
    # Format neighborhood names
    df["bairro"] = df["bairro"].apply(lambda x: unidecode(x.lower().strip()))

    # Drop duplicates
    df = df.drop_duplicates(subset=["id"])
    
    # Return dataframe
    return df

In [4]:
# Create duckdb connection
con = duckdb.connect(database=r"C:\Users\PedroMiyasaki\OneDrive - DHAUZ\Área de Trabalho\Projetos\PESSOAL\kodomiya\kodomiya\db\kodomiya.duckdb")

# Define kb queries
kb_querys = [
    "SELECT * FROM kodomiya_chaves_na_mao.chaves_na_mao_register",
    "SELECT * FROM kodomiya_viva_real.viva_real_register",
    "SELECT * FROM kodomiya_zap_imoveis.zap_imoveis_register"
]

# Concatenate all kb dataframes
df_kb = pd.concat([con.execute(query).fetch_df() for query in kb_querys], ignore_index=True)

# Do data-cleaning
df_kb = kb_data_cleaning(df_kb)

# Read leilao data
df_leilao = con.execute("SELECT * FROM kodomiya_leilao_imovel.leilao_imovel_register").fetch_df()

# Clean leilao data
df_leilao = leilao_data_cleaning(df_leilao)

# Filter properties that acept financial credit
df_leilao = df_leilao.loc[df_leilao["aceita_financiamento"]]

# Make columns of price per m2
df_leilao["preco_m2_area_util"] = df_leilao["preco_atual"] / df_leilao["area_util"]
df_leilao["preco_m2_area_total"] = df_leilao["preco_atual"] / df_leilao["area_terreno"]
df_kb["preco_m2"] = df_kb["preco"] / df_kb["tamanho"]

# Calculate mean and std of price per m2 of df_kb by neighborhood (also make a more general mean and std)
df_kb_mean_price_per_m2_by_neighborhood = df_kb.groupby("bairro")["preco_m2"].agg(["mean", "std"]).rename(columns={"mean": "mean_preco_m2_by_neighborhood", "std": "std_preco_m2_by_neighborhood"})
df_kb_mean_price_per_m2_general = df_kb["preco_m2"].agg(["mean", "std"])

# Bring price per m2 of df_kb_mean_price_per_m2 to df_leilao
df_leilao = df_leilao.merge(df_kb_mean_price_per_m2_by_neighborhood, left_on="bairro", right_on="bairro", how="left")
df_leilao["mean_preco_m2_general"] = df_kb_mean_price_per_m2_general["mean"]
df_leilao["std_preco_m2_general"] = df_kb_mean_price_per_m2_general["std"]

# Calculate z-score for price per m2
df_leilao["z_score_preco_m2_by_neighborhood_area_util"] = (df_leilao["preco_m2_area_util"] - df_leilao["mean_preco_m2_by_neighborhood"]) / df_leilao["std_preco_m2_by_neighborhood"]
df_leilao["z_score_preco_m2_general_area_util"] = (df_leilao["preco_m2_area_util"] - df_leilao["mean_preco_m2_general"]) / df_leilao["std_preco_m2_general"]
df_leilao["z_score_preco_m2_by_neighborhood_area_total"] = (df_leilao["preco_m2_area_total"] - df_leilao["mean_preco_m2_by_neighborhood"]) / df_leilao["std_preco_m2_by_neighborhood"]
df_leilao["z_score_preco_m2_general_area_total"] = (df_leilao["preco_m2_area_total"] - df_leilao["mean_preco_m2_general"]) / df_leilao["std_preco_m2_general"]

# Calculate mean of z-scores for each row, ignoring NaN values
df_leilao["mean_z_score"] = df_leilao[["z_score_preco_m2_by_neighborhood_area_util", 
                                      "z_score_preco_m2_general_area_util",
                                      "z_score_preco_m2_by_neighborhood_area_total", 
                                      "z_score_preco_m2_general_area_total"]].mean(axis=1, skipna=True)

# Filter at least one negative z-score
df_leilao = df_leilao[
    (df_leilao["z_score_preco_m2_by_neighborhood_area_util"] < 0) |
    (df_leilao["z_score_preco_m2_general_area_util"] < 0) |
    (df_leilao["z_score_preco_m2_by_neighborhood_area_total"] < 0) |
    (df_leilao["z_score_preco_m2_general_area_total"] < 0)
]

# Count how many z-score columns are filled (not NaN) for each row
df_leilao["confidence"] = df_leilao[["z_score_preco_m2_by_neighborhood_area_util", 
                                          "z_score_preco_m2_general_area_util",
                                          "z_score_preco_m2_by_neighborhood_area_total", 
                                          "z_score_preco_m2_general_area_total"]].notna().sum(axis=1)


# Create rank columns based on cheap_confidence (descending) and mean_z_score (ascending)
df_leilao["rank_confidence"] = df_leilao["confidence"].rank(ascending=False, method="min")
df_leilao["rank_mean_z_score"] = df_leilao["mean_z_score"].rank(ascending=True, method="min")

# Calculate final score
df_leilao["final_score"] = df_leilao["rank_confidence"] + df_leilao["rank_mean_z_score"]

# Order by final_score
df_leilao = df_leilao.sort_values(by="final_score", ascending=True)

# Return max m2 column
df_leilao['max_mean_preco_m2'] = df_leilao[['mean_preco_m2_by_neighborhood', 'mean_preco_m2_general']].max(axis=1)

# Multiply this by area
df_leilao["expected_value"] = df_leilao["max_mean_preco_m2"] * df_leilao["area_util"]

# Calculte ROI
df_leilao["fake_roi"] = (df_leilao["expected_value"] - df_leilao["preco_atual"]) / df_leilao["preco_atual"]

# Filter at least 15% of ROI
df_leilao = df_leilao[df_leilao["fake_roi"] >= 0.15]

In [6]:
df_kb_mean_price_per_m2_by_neighborhood 

,mean_preco_m2_by_neighborhood,std_preco_m2_by_neighborhood
bairro,,
abranches,5100.605047,1871.510283
agua verde,9330.758029,4244.995198
ahu,11046.677669,3107.043207
alto boqueirao,5110.946337,1411.237872
alto da gloria,11020.357232,4351.975673
...,...,...
uberaba,5753.903876,1653.440441
umbara,5777.319407,1341.695335
vila izabel,8884.220633,2599.984550


In [7]:
df_leilao

,id,datahora,preco_primeira_praca,data_primeira_praca,preco_segunda_praca,data_segunda_praca,preco_atual,area_util,area_terreno,rua,...,z_score_preco_m2_by_neighborhood_area_total,z_score_preco_m2_general_area_total,mean_z_score,confidence,rank_confidence,rank_mean_z_score,final_score,max_mean_preco_m2,expected_value,fake_roi
1,8bbcf92a8fb2e8e662a8debf4710905c,2025-06-02 15:03:35.668985-03:00,460000.00,2025-07-08 07:00:00-03:00,276000.00,2025-07-15 07:00:00-03:00,460000.00,89.50,180.00,RUA WALDEMAR BONA,...,-1.858090,-1.170141,-0.955528,4,1.0,1.0,2.0,7173.550473,642032.767304,0.395723
7,10f25bb0b52259d56bbb96e1442edf9d,2025-06-02 15:02:49.670035-03:00,370000.00,2025-06-03 07:00:00-03:00,322315.00,2025-06-10 07:00:00-03:00,370000.00,92.94,88.20,RUA GUARANA,...,NaN,-0.754724,-0.781830,2,3.0,2.0,5.0,7173.550473,666709.780930,0.801918
3,c70ba8bdf53b5192226afd4e21b4e760,2025-06-02 15:02:48.688342-03:00,430000.00,2025-06-03 07:00:00-03:00,811354.40,2025-06-10 07:00:00-03:00,430000.00,93.05,84.85,RUA BOM SUCESSO,...,NaN,-0.533579,-0.590160,2,3.0,3.0,6.0,7173.550473,667498.871482,0.552323
4,dad8362e15d38981090b7d003c10764e,2025-06-02 15:03:55.798682-03:00,420000.00,2025-07-21 07:00:00-03:00,252000.00,2025-07-24 07:00:00-03:00,420000.00,83.81,80.40,RUA PEDRO AIRTON ZIMMERMANN,...,NaN,-0.494022,-0.520950,2,3.0,4.0,7.0,7173.550473,601215.265115,0.431465
5,10efc127510b91b7a3ba780ff2bf74d3,2025-06-02 15:03:56.728107-03:00,360841.20,2025-07-21 07:00:00-03:00,334488.59,2025-07-24 07:00:00-03:00,360841.20,72.00,72.00,TRAVESSA FRANCISCA PAMPUCHE ANTONIACOMI,...,0.584623,-0.547790,0.018417,4,1.0,7.0,8.0,7173.550473,516495.634032,0.431365
2,613746f55cf22a087dc117111cc5f3f9,2025-06-02 15:03:06.656790-03:00,354758.18,2025-06-16 07:00:00-03:00,215868.67,2025-06-25 07:00:00-03:00,354758.18,63.43,NaN,RUA OSCAR GOMES DE OLIVEIRA,...,NaN,NaN,-0.247535,2,3.0,5.0,8.0,7173.550473,455018.306482,0.282615


In [80]:
df_leilao[["link", "id", "preco_atual", "bairro", "rua", "final_score", "fake_roi", "expected_value"]]

,id,datahora,preco_primeira_praca,data_primeira_praca,preco_segunda_praca,data_segunda_praca,preco_atual,area_util,area_terreno,rua,...,z_score_preco_m2_by_neighborhood_area_total,z_score_preco_m2_general_area_total,mean_z_score,confidence,rank_confidence,rank_mean_z_score,final_score,max_mean_preco_m2,expected_value,roi
1,8bbcf92a8fb2e8e662a8debf4710905c,2025-06-02 15:03:35.668985-03:00,460000.00,2025-07-08 07:00:00-03:00,276000.00,2025-07-15 07:00:00-03:00,460000.00,89.50,180.00,RUA WALDEMAR BONA,...,-1.858090,-1.170141,-0.955528,4,1.0,1.0,2.0,7173.550473,642032.767304,0.395723
7,10f25bb0b52259d56bbb96e1442edf9d,2025-06-02 15:02:49.670035-03:00,370000.00,2025-06-03 07:00:00-03:00,322315.00,2025-06-10 07:00:00-03:00,370000.00,92.94,88.20,RUA GUARANA,...,NaN,-0.754724,-0.781830,2,3.0,2.0,5.0,7173.550473,666709.780930,0.801918
3,c70ba8bdf53b5192226afd4e21b4e760,2025-06-02 15:02:48.688342-03:00,430000.00,2025-06-03 07:00:00-03:00,811354.40,2025-06-10 07:00:00-03:00,430000.00,93.05,84.85,RUA BOM SUCESSO,...,NaN,-0.533579,-0.590160,2,3.0,3.0,6.0,7173.550473,667498.871482,0.552323
4,dad8362e15d38981090b7d003c10764e,2025-06-02 15:03:55.798682-03:00,420000.00,2025-07-21 07:00:00-03:00,252000.00,2025-07-24 07:00:00-03:00,420000.00,83.81,80.40,RUA PEDRO AIRTON ZIMMERMANN,...,NaN,-0.494022,-0.520950,2,3.0,4.0,7.0,7173.550473,601215.265115,0.431465
5,10efc127510b91b7a3ba780ff2bf74d3,2025-06-02 15:03:56.728107-03:00,360841.20,2025-07-21 07:00:00-03:00,334488.59,2025-07-24 07:00:00-03:00,360841.20,72.00,72.00,TRAVESSA FRANCISCA PAMPUCHE ANTONIACOMI,...,0.584623,-0.547790,0.018417,4,1.0,7.0,8.0,7173.550473,516495.634032,0.431365
2,613746f55cf22a087dc117111cc5f3f9,2025-06-02 15:03:06.656790-03:00,354758.18,2025-06-16 07:00:00-03:00,215868.67,2025-06-25 07:00:00-03:00,354758.18,63.43,NaN,RUA OSCAR GOMES DE OLIVEIRA,...,NaN,NaN,-0.247535,2,3.0,5.0,8.0,7173.550473,455018.306482,0.282615


# Testar scrapers

## Leilao imovel

ImportError: attempted relative import with no known parent package

In [7]:
import cloudscraper

url = "https://www.leilaoimovel.com.br/banco_leilao_de_imoveis/caixa-economica-federal-cef?s=&tipo=2,1&cidade=4106902&pagamento=3,1&preco_max=500000&ordem=dt_auction_a&pag=2"

# Create a scraper instance
scraper = cloudscraper.create_scraper()
response = scraper.get(url)

In [3]:
from bs4 import BeautifulSoup

In [11]:
soup = BeautifulSoup(response.content, "html.parser")

In [12]:
cards_imoveis = soup.find_all("div", class_="place-box")

In [16]:
cards_imoveis[-1]

<div class="place-box" style="display: none;">
<div class="image">
<div class="place-tag">
<span><b>Novo!</b></span>
</div>
<a class="Link_Redirecter" href="/imoveis-springfield" target="">
<picture class="d-flex">
<source srcset="https://image.leilaoimovel.com.br/images/property/imovel-sem-foto.webp"/>
<img alt="" loading="lazy" src="https://image.leilaoimovel.com.br/images/property/imovel-sem-foto.webp" title=""/>
</picture>
</a>
<div class="tag">
<object alt="Lance imperdível" class="svg me-2" data="https://image.leilaoimovel.com.br/img/li/ri_auction-line.svg" height="24" title="Lance imperdível" type="image/svg+xml" width="24"></object>
<span>Data de encerramento: 05/05/2023 08:00</span>
</div>
</div>
<div class="categories">
<a href="/imoveis-springfield"></a>
</div>
<a class="Link_Redirecter" href="/imoveis-springfield" target="">
<div class="prices">
<div class="price">
<div>
<span class="discount-price font-1">
                        R$ 262.500,00
                    </span>
<

## Imports

In [3]:
# Imports DLT
import dlt
from dlt.sources.helpers import requests

# Imports online
from geopy.geocoders import Nominatim
from geopy.point import Point
from datetime import datetime
from typing import Iterable
from requests.exceptions import HTTPError
from bs4 import BeautifulSoup
import logging
import sys

In [5]:
import cloudscraper

url = "https://www.vivareal.com.br/venda/parana/curitiba/apartamento_residencial/?transacao=venda&onde=,Paran%C3%A1,Curitiba,,,,,city,BR%3EParana%3ENULL%3ECuritiba,-25.426899,-49.265198,&tipos=apartamento_residencial,casa_residencial,condominio_residencial,cobertura_residencial,flat_residencial,kitnet_residencial,sobrado_residencial&pagina=2"

# Create a scraper instance
scraper = cloudscraper.create_scraper()

try:
    response = scraper.get(url)
    response.raise_for_status() # Will raise an exception for 4xx/5xx errors
    print("Request successful!")
    # print(response.text) # Access the content
    # print(response.status_code)
except requests.exceptions.HTTPError as err:
    print(f"HTTP error occurred: {err}")
    print(f"Status Code: {err.response.status_code}")
    print(f"Response text: {err.response.text[:500]}") # Print first 500 chars of response
except Exception as err:
    print(f"An error occurred: {err}")

Request successful!


In [4]:
response = requests.get("https://www.vivareal.com.br/venda/parana/curitiba/apartamento_residencial/?transacao=venda&onde=,Paran%C3%A1,Curitiba,,,,,city,BR%3EParana%3ENULL%3ECuritiba,-25.426899,-49.265198,&tipos=apartamento_residencial,casa_residencial,condominio_residencial,cobertura_residencial,flat_residencial,kitnet_residencial,sobrado_residencial", allow_redirects=False)
soup = BeautifulSoup(response.content, "html.parser")
#cards_imoveis = soup.find_all("div", class_="flex flex-col grow min-w-0 content-stretch border-neutral-90 min-[1280px]:border-l pb-2 gap-2")

HTTPError: 403 Client Error: Forbidden for url: https://www.vivareal.com.br/venda/parana/curitiba/apartamento_residencial/?transacao=venda&onde=,Paran%C3%A1,Curitiba,,,,,city,BR%3EParana%3ENULL%3ECuritiba,-25.426899,-49.265198,&tipos=apartamento_residencial,casa_residencial,condominio_residencial,cobertura_residencial,flat_residencial,kitnet_residencial,sobrado_residencial

In [11]:
cards_imoveis[0]

<div class="flex flex-col grow min-w-0 content-stretch border-neutral-90 min-[1280px]:border-l pb-2 gap-2"><div class="min-h-8 border-neutral-90 px-2 py-1-5 text-nowrap text-ellipsis overflow-hidden flex flex-row items-start"><div class="flex flex-col items-stretch grow overflow-hidden text-nowrap"><h2 class="text-ellipsis text-2 font-semibold overflow-hidden font-secondary" data-cy="rp-cardProperty-location-txt"><span class="block font-secondary text-1-5 font-regular text-neutral-110 mb-1">Apartamento para comprar em<!-- --> </span>Alto da Glória, Curitiba</h2><p class="text-1-75 font-regular text-ellipsis overflow-hidden" data-cy="rp-cardProperty-street-txt">Avenida João Gualberto</p></div><button aria-checked="false" class="olx-core-button olx-core-button--tertiary olx-core-button--medium olx-core-button--only-icon hidden md:block" role="switch"><svg aria-hidden="true" data-testid="heart" height="24" viewbox="0 0 24 24" width="24" xmlns="http://www.w3.org/2000/svg"><path d="M20.1410

### Chaves na mao

In [29]:
# Fazer tupla com sinonimos de rua
street_synonyms = (
    "rua",
    "avenida",
    "alameda",
    "travessa",
    "praça",
    "largo",
    "viela",
    "estrada",
    "boulevard",
    "passagem",
    "via",
    "rodovia"
)

# Fazer tupla com nomes de cidades brasileiras
city_names = (
    "curitiba",
    "porto belo",
)

# Fazer tupla com o nome de bairros
neighborhood_names = (
    "abatia",
    "abranches",
    "agua verde",
    "ahu",
    "alto boqueirao",
    "alto da gloria",
    "alto da rua xv",
    "alto do boa vista",
    "alto da xv",
    "atuba",
    "augusta",
    "bacacheri",
    "bairros novos",
    "barreirinha",
    "batel",
    "bigorrilho",
    "boa vista",
    "bom retiro",
    "boqueirao",
    "butiatuvinha",
    "cabral",
    "cachoeira",
    "cajuru",
    "campina do siqueira",
    "campo comprido",
    "capao da imbuia",
    "capao raso",
    "cascatinha",
    "centro",
    "centro civico",
    "cic",
    "cidade industrial",
    "cristo rei",
    "fanny",
    "fazendinha",
    "ganchinho",
    "guabirotuba",
    "guaira",
    "hauer",
    "hugo lange",
    "jardim botanico",
    "jardim das americas",
    "jardim social",
    "juveve",
    "lamenha pequena",
    "lindoia",
    "merces",
    "mossungue",
    "novo mundo",
    "orleans",
    "parolin",
    "pilarzinho",
    "pinheirinho",
    "porto",
    "prado velho",
    "reboucas",
    "santa candida",
    "santa felicidade",
    "santa quiteria",
    "santo inacio",
    "sao braz",
    "sao francisco",
    "sao joao",
    "sao lourenco",
    "sao miguel",
    "sao sebastiao",
    "taboao",
    "taruma",
    "tatuquara",
    "tingui",
    "uberaba",
    "umbara",
    "vila izabel",
    "vista alegre",
    "xaxim"
)



In [30]:
# Imports online
from unidecode import unidecode
from hashlib import md5

# Fazer função que só deixa numeros em uma string
def return_only_alphanumeric_part(word):
    # Retornar apenar parte numérica da palavra
    return "".join([c for c in word if c.isnumeric()])

# Fazer função de achar palavra em uma sentença e retornar palavra
def return_word_founded_in_sentence(sentence, list_of_words, lower_sentence=True):
    # Aplicar unidecode na frase
    sentence = unidecode(sentence)
    
    # Converter frase para lower se lower_sentence true
    if lower_sentence:
        sentence = sentence.lower()

    # Iterar a lista de palavras
    for word in list_of_words:
        if word in sentence:
            return word
        
    # Do contrário, retorno Nulo
    return None

# Fazer função para geração de id com sequencia de plaavars
def make_propertie_id(list_of_string_to_concatenate):
    # Retirar nulos da lista caso eles existam
    cleaned_list_of_string_to_concatenate = [i for i in list_of_string_to_concatenate if i is not None]

    # Montar string para o id
    id_string = "".join(cleaned_list_of_string_to_concatenate)

    # Passar limpeza no id
    id_string = unidecode(id_string.lower().strip().replace(" ", "").replace(",", ""))

    # Gerar hash md5 com string
    return md5(id_string.encode("utf-8")).hexdigest()

In [31]:
class chavesNaMao():
    # Fazer função para retornar o preço no site chaves na mao
    @staticmethod
    def return_chaves_na_mao_preco(propertie_card, price_tag, price_class_name, price_value_tag, **kwargs):
        # Buscar preço no card da propriedade
        try:
            price_element = propertie_card.find(price_tag, class_=price_class_name)
            price_text = price_element.find(price_value_tag).text
            price_text = price_text.replace("R$ ", "").replace(".", "_")
            
            # Tentar retornar o preço convertido para float
            return float(price_text)
        
        except (AttributeError, ValueError, IndexError) as e:
            return None
        
    # Fazer função para retornar o tamanho do imóvel no site chaves na mao
    @staticmethod
    def return_chaves_na_mao_tamanho(propertie_card, tag, class_name, index=0, split_text=None, **kwargs):
        # Buscar pelo tamanho
        try:
            # Find all elements with the specified tag and class
            tamanho_elements = propertie_card.find_all(tag, class_=class_name)
            
            if tamanho_elements and len(tamanho_elements) > index:
                tamanho_text = tamanho_elements[index].text.strip()
                
                # Split by the specified text if provided
                if split_text and split_text in tamanho_text:
                    tamanho = tamanho_text.split(split_text)[0].strip()
                else:
                    tamanho = tamanho_text
                
                # Retornar tamanho convertido para inteiro
                return int(tamanho)
            return None
        except (ValueError, IndexError):
            return None

    # Fazer função para retornar o n de quartos no site chaves na mao
    @staticmethod
    def return_chaves_na_mao_n_quartos(propertie_card, tag, class_name, search_text, **kwargs):
        try:
            # Procurar nos elementos com a tag e classe especificadas que contém o texto de busca
            room_elements = [p for p in propertie_card.find_all(tag, class_=class_name) 
                           if search_text in p.text]
            
            if room_elements:
                # Extrair somente o número dos quartos
                room_text = room_elements[0].text.strip()
                # Pegar o número antes do texto de busca
                room_number = room_text.split(search_text)[0].strip()
                return int(return_only_alphanumeric_part(room_number))
            return 0
        except (ValueError, IndexError):
            return 0
        
    # Fazer função para retornar o n de banheiros no site chaves na mao
    @staticmethod
    def return_chaves_na_mao_n_banheiros(propertie_card, tag, class_name, search_text, **kwargs):
        try:
            # Procurar nos elementos com a tag e classe especificadas que contém o texto de busca
            bathroom_elements = [p for p in propertie_card.find_all(tag, class_=class_name) 
                               if search_text in p.text]
            
            if bathroom_elements:
                # Extrair somente o número dos banheiros
                bathroom_text = bathroom_elements[0].text.strip()
                # Pegar o número antes do texto de busca
                bathroom_number = bathroom_text.split(search_text)[0].strip()
                return int(return_only_alphanumeric_part(bathroom_number))
            return 0
        except (ValueError, IndexError):
            return 0
        
    # Fazer função para retornar o n de garagems no site chaves na mao
    @staticmethod
    def return_chaves_na_mao_n_vagas_garagem(propertie_card, tag, class_name, search_text, **kwargs):
        try:
            # Procurar nos elementos com a tag e classe especificadas que contém o texto de busca
            garage_elements = [p for p in propertie_card.find_all(tag, class_=class_name) 
                             if search_text in p.text]
            
            if garage_elements:
                # Extrair somente o número das garagens
                garage_text = garage_elements[0].text.strip()
                # Pegar o número antes do texto de busca
                garage_number = garage_text.split(search_text)[0].strip()
                return int(return_only_alphanumeric_part(garage_number))
            return 0
        except (ValueError, IndexError):
            return 0
        
    # Fazer função para retornar o endereço no site chaves na mao
    @staticmethod
    def return_chaves_na_mao_endereco(propertie_card, main_tag, class_name, rua_tag, rua_index, bairro_cidade_tag, bairro_cidade_index, **kwargs):
        try:
            # Buscar a tag de endereço
            address_element = propertie_card.find(main_tag, class_=class_name)
            
            if address_element:
                # Extrair rua do endereco (primeiro elemento p)
                p_elements = address_element.find_all(rua_tag)
                rua = p_elements[rua_index].text.strip() if len(p_elements) > rua_index else ""
                
                # Extrair bairro e cidade do segundo elemento p
                bairro_e_cidade = p_elements[bairro_cidade_index].text.strip() if len(p_elements) > bairro_cidade_index else ""
                
                # Extrair bairro do endereço
                bairro = return_word_founded_in_sentence(bairro_e_cidade, neighborhood_names)
                
                # Extrair cidade do endereço
                cidade = return_word_founded_in_sentence(bairro_e_cidade, city_names)
                
                return rua, bairro, cidade
            
            return "", "", ""
        except (IndexError, AttributeError):
            return "", "", ""

In [32]:
c = chavesNaMao()

In [74]:
response = requests.get("https://www.chavesnamao.com.br/imoveis-a-venda/pr-curitiba/?pg=2", allow_redirects=False)
soup = BeautifulSoup(response.content, "html.parser")
cards_imoveis = soup.find_all("span", class_="card-module__1awNxG__cardContent")

In [75]:
chavesNaMao.return_chaves_na_mao_preco(
    cards_imoveis[0], 
    "p", 
    "column style-module__PkTDxW__price",
    "b",
)

400000.0

In [40]:
cards_imoveis[0]

<span class="card-module__1awNxG__cardContent"><p aria-label="Preço" class="column style-module__PkTDxW__price"><b>R$ 798.000</b><small aria-label="IPTU">Iptu: <!-- -->R$ 1.812</small></p><span aria-label="list" class="style-module__PkTDxW__list"><p aria-label="" class="styles-module__5TXV2W__body2 undefined"> <!-- -->149<!-- -->m²</p><p aria-label="" class="styles-module__5TXV2W__body2 undefined"><i></i>4<!-- --> Quartos</p><p aria-label="" class="styles-module__5TXV2W__body2 undefined"><i></i>2<!-- --> Garagem</p><p aria-label="" class="styles-module__5TXV2W__body2 undefined"><i></i>1<!-- --> Banheiro</p></span><span class="style-module__PkTDxW__content"><h2 class="styles-module__5TXV2W__heading2 styles-module__saqrOW__title undefined">Sobrado em SANTA FELICIDADE com 149,34 m²</h2><p class="styles-module__5TXV2W__body1">Sobrado com Jardim em Ótima Localização! Descubra o conforto e a praticidade deste otimo sobrado de 149m², ideal para sua família! 4 quartos (sendo 1 ...</p></span><a

In [44]:
cards_imoveis[0].find("p", class_="column style-module__PkTDxW__price")

<p aria-label="Preço" class="column style-module__PkTDxW__price"><b>R$ 798.000</b><small aria-label="IPTU">Iptu: <!-- -->R$ 1.812</small></p>

In [35]:
cards_imoveis[0]

<span class="card-module__1awNxG__cardContent"><p aria-label="Preço" class="column style-module__PkTDxW__price"><b>R$ 798.000</b><small aria-label="IPTU">Iptu: <!-- -->R$ 1.812</small></p><span aria-label="list" class="style-module__PkTDxW__list"><p aria-label="" class="styles-module__5TXV2W__body2 undefined"> <!-- -->149<!-- -->m²</p><p aria-label="" class="styles-module__5TXV2W__body2 undefined"><i></i>4<!-- --> Quartos</p><p aria-label="" class="styles-module__5TXV2W__body2 undefined"><i></i>2<!-- --> Garagem</p><p aria-label="" class="styles-module__5TXV2W__body2 undefined"><i></i>1<!-- --> Banheiro</p></span><span class="style-module__PkTDxW__content"><h2 class="styles-module__5TXV2W__heading2 styles-module__saqrOW__title undefined">Sobrado em SANTA FELICIDADE com 149,34 m²</h2><p class="styles-module__5TXV2W__body1">Sobrado com Jardim em Ótima Localização! Descubra o conforto e a praticidade deste otimo sobrado de 149m², ideal para sua família! 4 quartos (sendo 1 ...</p></span><a

In [ ]:
preco = chavesNaMao.return_chaves_na_mao_preco(
    cards_imoveis[0], 
    CHAVES_CONFIG['price']['tag'], 
    CHAVES_CONFIG['price']['class_name'],
    CHAVES_CONFIG['price']['price_value_tag'],
)

TypeError: chavesNaMao.return_chaves_na_mao_tamanho() missing 2 required positional arguments: 'tag' and 'class_name'

In [49]:
# Display the HTML content of the first card as text
html_content = str(cards_imoveis[0])
for line in html_content.split(">"):
    if line.strip():
        print(line + ">")

<span class="card-module__cvK-Xa__cardContent">
<p aria-label="Preço" class="column style-module__Yo5w-q__price">
<b>
R$ 646.000</b>
</p>
<span aria-label="list" class="style-module__Yo5w-q__list">
<p aria-label="" class="styles-module__aBT18q__body2 undefined">
 <!-- -->
105<!-- -->
m²</p>
<p aria-label="" class="styles-module__aBT18q__body2 undefined">
<i>
</i>
3<!-- -->
 Quartos</p>
<p aria-label="" class="styles-module__aBT18q__body2 undefined">
<i>
</i>
2<!-- -->
 Garagem</p>
<p aria-label="" class="styles-module__aBT18q__body2 undefined">
<i>
</i>
3<!-- -->
 Banheiros</p>
</span>
<span class="style-module__Yo5w-q__content">
<h2 class="styles-module__aBT18q__heading2 styles-module__ViVk2q__title undefined">
Casa com 3 quartos à venda na Rua Rio Mucuri, 1378, Bairro Alto, Curitiba por R$ 646.000</h2>
<p class="styles-module__aBT18q__body1">
SOBRADO EM CONDOMÍNIO NO BAIRRO ALTO Sobrado com 105m² com excelentes acabamentos Pavimento Térreo: - Sala de estar/ jantar - Cozinha em concei

# Carga duckdb

# Funções

In [16]:
def find_best_n_of_clusters(dataframe, columns, cluster_range):
    """
    Perform clustering using KMeans and find the best number of clusters based on silhouette score.

    Parameters:
    - dataframe: DataFrame containing the dataframe for clustering.
    - columns: columns in the DataFrame to use for clustering.
    - cluster_range: Range of cluster numbers to consider.

    Returns:
    - best_clusters: Number of clusters that yield the highest silhouette score.
    - cluster_labels: Cluster labels assigned by KMeans.
    """

    # Extract the feature for clustering
    features = dataframe[columns]

    # Scale feature befores clustering
    mms = MinMaxScaler()
    features = mms.fit_transform(dataframe[columns])

    # Initialize variables to store the best silhouette score and corresponding number of clusters + history of seacrh
    best_score = -1
    best_clusters = 0
    cluster_labels = None
    n_clusters_history = {"n_clusters": [], "score": []}

    # Iterate over the specified range of cluster numbers
    for n_clusters in cluster_range:
        # Veririficar se o número de cluster testados da iteração é maior que o numero de amostras
        if n_clusters >= len(features):
            break # Se for, quebre o loop

        # Create a KMeans model with the current number of clusters
        kmeans = KMeans(n_clusters=n_clusters, random_state=32, n_init="auto")
        
        # Fit the model and get cluster labels
        labels = kmeans.fit_predict(features)
        
        # Calculate silhouette score
        score = silhouette_score(features, labels)

        # Save current atempt at n_clusters_history
        n_clusters_history["n_clusters"].append(n_clusters)
        n_clusters_history["score"].append(score)
        
        # Update best score and number of clusters if the current result is better
        if score > best_score:
            best_score = score
            best_clusters = n_clusters
            cluster_labels = labels

    print(f"Best Number of Clusters: {best_clusters}")
    print(f"Silhouette Score: {best_score}")

    return best_clusters, cluster_labels, n_clusters_history

# Fazer função de retorno de statisticas por cluster
def return_cluster_agg_stats(dataframe, cluster_column):
    # Retornar dataframe agregado
    return dataframe.groupby(cluster_column).agg({
        col: ["mean", "min", "max", "std"] for col in dataframe.select_dtypes("number").columns if col != cluster_column
    })


# Forecasting

In [22]:
# Ler excel fipe zap com pandas (Curitiba)
df = pd.read_excel("../data/fipezap-serieshistoricas.xlsx", sheet_name="Curitiba", header=[0, 3], index_col=1)

# Selecionar colunas de preço médio do ímovel residencial vendido em Curitiba
df = df[[
    ('Imóveis residenciais', 'Total.8'),
    ('Imóveis residenciais',    '1D.8'),
    ('Imóveis residenciais',    '2D.8'),
    ('Imóveis residenciais',    '3D.8'),
    ('Imóveis residenciais',    '4D.8')
    ]]

# Dropar um index
df = df.droplevel(level=None)

KeyError: 'Requested level (Imóveis residenciais) does not match index name (None)'

In [27]:
df.droplevel(level="Curitiba")

KeyError: 'Requested level (Curitiba) does not match index name (None)'

Curitiba                                           Imóveis residenciais       \
Data                                                            Total.8 1D.8   
2008-01-01 00:00:00                                                   .    .   
2008-02-01 00:00:00                                                   .    .   
2008-03-01 00:00:00                                                   .    .   
2008-04-01 00:00:00                                                   .    .   
2008-05-01 00:00:00                                                   .    .   
...                                                                 ...  ...   
NaN                                                                 NaN  NaN   
Nota: Os preços considerados para o cálculo são...                  NaN  NaN   
Fonte: ZAP e IBGE.                                                  NaN  NaN   
Note: Monthly prices are 3-months moving averages.                  NaN  NaN   
Source: ZAP and IBGE.                                               NaN  NaN   

Curitiba                                                           
Data                                               2D.8 3D.8 4D.8  
2008-01-01 00:00:00                                   .    .    .  
2008-02-01 00:00:00                                   .    .    .  
2008-03-01 00:00:00                                   .    .    .  
2008-04-01 00:00:00                                   .    .    .  
2008-05-01 00:00:00                                   .    .    .  
...                                                 ...  ...  ...  
NaN                                                 NaN  NaN  NaN  
Nota: Os preços considerados para o cálculo são...  NaN  NaN  NaN  
Fonte: ZAP e IBGE.                                  NaN  NaN  NaN  
Note: Monthly prices are 3-months moving averages.  NaN  NaN  NaN  
Source: ZAP and IBGE.                               NaN  NaN  NaN  

[197 rows x 5 columns]

In [122]:
df.columns

MultiIndex([(            'Curitiba',    'Data'),
            ('Imóveis residenciais',   'Total'),
            ('Imóveis residenciais',      '1D'),
            ('Imóveis residenciais',      '2D'),
            ('Imóveis residenciais',      '3D'),
            ('Imóveis residenciais',      '4D'),
            ('Imóveis residenciais', 'Total.1'),
            ('Imóveis residenciais',    '1D.1'),
            ('Imóveis residenciais',    '2D.1'),
            ('Imóveis residenciais',    '3D.1'),
            ('Imóveis residenciais',    '4D.1'),
            ('Imóveis residenciais', 'Total.2'),
            ('Imóveis residenciais',    '1D.2'),
            ('Imóveis residenciais',    '2D.2'),
            ('Imóveis residenciais',    '3D.2'),
            ('Imóveis residenciais',    '4D.2'),
            ('Imóveis residenciais', 'Total.3'),
            ('Imóveis residenciais',    '1D.3'),
            ('Imóveis residenciais',    '2D.3'),
            ('Imóveis residenciais',    '3D.3'),
            ('Imóvei

In [110]:
df[('Imóveis residenciais', 'Total')]

NaN      .
NaN      .
NaN      .
NaN      .
NaN      .
      ... 
NaN    NaN
NaN    NaN
NaN    NaN
NaN    NaN
NaN    NaN
Name: (Imóveis residenciais, Total), Length: 197, dtype: object

# Clusterização

## Carregar dados

In [18]:
# Criar conexão
con = duckdb.connect(database=r"C:\Users\PedroMiyasaki\OneDrive - DHAUZ\Área de Trabalho\Projetos\PESSOAL\trading-properties\db\trading_properties_db.duckdb")

# Carregar registro chaves na mão
df_chaves_na_mao = con.execute("""
    SELECT 
        * 
    FROM trading_properties_db.chaves_na_mao_schema.chaves_na_mao_register
""").fetch_df()
df_chaves_na_mao["fonte_dado"] = "Chaves na Mão"

# Carregar registro zap imoveis
df_zap_imoveis = con.execute("""
    SELECT 
        * 
    FROM trading_properties_db.zap_imoveis_schema.zap_imoveis_register
""").fetch_df()
df_zap_imoveis["fonte_dado"] = "Zap Imóveis"

# Carregar registro viva real
df_viva_real = con.execute("""
    SELECT 
        * 
    FROM trading_properties_db.viva_real_schema.viva_real_register
""").fetch_df()
df_viva_real["fonte_dado"] = "Viva Real"

# Fechar conexão
con.close()

# Contenar dataframe
df = pd.concat([df_chaves_na_mao, df_zap_imoveis, df_viva_real], ignore_index=True)

# Dropar linhas com ids de imóveis iguais
df = df.sort_values("datahora").drop_duplicates("id", keep="last")

# Fazer um id com a latitude e longitude específica
df["lat_long_id"] = df["latitude"].apply(str) + "-" + df["longitude"].apply(str)

# Contar o numero de acontecimentos daquela latitude e longitude
df["count_lat_log_id"] = df.groupby("lat_long_id")["id"].transform("count")

# Transformar latitudes e longitudes repetidas em nulas
df.loc[df["count_lat_log_id"] > 1, "latitude"] = None
df.loc[df["count_lat_log_id"] > 1, "longitude"] = None

# Retirar linhas de preco ou tamanho nulo
df = df.loc[ ( ~ df["tamanho"].isna()) & ( ~ df["preco"].isna()) ]

# Retirar outliers de preço ou tamanho execivos (3 Z-scores)
df = df[(np.abs(stats.zscore(df[["tamanho", "preco"]])) < 3).all(axis=1)]

# Rodar um processo de clusterização GERAL
_, cluster_labels, _ = find_best_n_of_clusters(df, ["tamanho", "n_banheiros", "n_quartos", "n_garagem"], range(2, 30))

# Definir coluna de clusterização geral
df["cluster_geral"] = cluster_labels

con.close()

Best Number of Clusters: 29
Silhouette Score: 0.4634284440640895


In [9]:
# Create a geolocator with a user agent
geolocator = Nominatim(user_agent="Trading Properties")

# Create the address by combining street, city, and state
df["endereco"] = df["rua"].str.strip() + " - " + df["cidade"].str.title() + " - PR"

# Set the bounding box for Curitiba
viewbox_curitiba = [Point(-25.717023, -49.674683), Point(-25.139069, -48.992157)]

# Apply geocoding with the viewbox parameter
df["geocode"] = df["endereco"].apply(lambda x: geolocator.geocode(x, viewbox=viewbox_curitiba, country_codes="BR", timeout=None, bounded=True))

## Filtro pré clusterização

In [75]:
# Retirar linhas de preco ou tamanho nulo
df = df.loc[ ( ~ df["tamanho"].isna()) & ( ~ df["preco"].isna()) ]

# Retirar outliers de preço ou tamanho execivos (3 Z-scores)
df = df[(np.abs(stats.zscore(df[["tamanho", "preco"]])) < 3).all(axis=1)]

## Clusterização

Rodar um processo geral, e um por bairro

In [76]:
# Rodar um processo de clusterização GERAL
best_clusters, cluster_labels, n_clusters_history = find_best_n_of_clusters(df, ["tamanho", "n_banheiros", "n_quartos", "n_garagem"], range(2, 30))

# Definir coluna de clusterização geral
df["cluster_geral"] = cluster_labels

# Instanciar lista para salver dataframes de bairro
bairro_dataframes = []

# Rodar um processo de clusterização por bairro
for bairro in df["bairro"].unique():
    # Filtrar apenas bairro da iteração
    df_bairro = df.loc[df["bairro"] == bairro].copy()

    # Rodar clusterização
    _, cluster_labels, _= find_best_n_of_clusters(df_bairro, ["tamanho", "n_banheiros", "n_quartos", "n_garagem"], range(2, 15))

    # Definir cluster do bairro
    df_bairro["cluster_bairro"] = cluster_labels

    # Guardar dataframe de bairro
    bairro_dataframes.append(df_bairro)

# Converter para dataframe
bairro_dataframe = pd.concat(bairro_dataframes, ignore_index=True)

# Juntar cluster de bairros ao df original
df = df.merge(bairro_dataframe[["id", "cluster_bairro"]], on="id", how="left")

# Marcar cluster de bairro "não encontrado" como nulo
df.loc[df["bairro"] == "não encontrado", "cluster_bairro"] = None

Best Number of Clusters: 29
Silhouette Score: 0.5004393617756344
Best Number of Clusters: 13
Silhouette Score: 0.5032646136049478
Best Number of Clusters: 2
Silhouette Score: 0.6017398248287854
Best Number of Clusters: 2
Silhouette Score: 0.47050082631655493
Best Number of Clusters: 14
Silhouette Score: 0.613357429474141
Best Number of Clusters: 14
Silhouette Score: 0.5401771242372764
Best Number of Clusters: 14
Silhouette Score: 0.4680716113819934
Best Number of Clusters: 2
Silhouette Score: 0.6139989743938624
Best Number of Clusters: 2
Silhouette Score: 0.6215974457428269
Best Number of Clusters: 12
Silhouette Score: 0.6809807363619735
Best Number of Clusters: 14
Silhouette Score: 0.44408089256038163
Best Number of Clusters: 14
Silhouette Score: 0.5491958344547324
Best Number of Clusters: 4
Silhouette Score: 0.5819528532956664
Best Number of Clusters: 2
Silhouette Score: 0.5749324572549916
Best Number of Clusters: 2
Silhouette Score: 0.4519469399668685
Best Number of Clusters: 3
Silh

In [79]:
# Fazer de preco medio cluster geral e por cluster bairro
df["preco_medio_cluster_geral"] = df.groupby("cluster_geral")["preco"].transform("mean")
df["preco_medio_cluster_bairro"] = df.groupby(["bairro", "cluster_bairro"])["preco"].transform("mean")

# Calcular dif % preço frente a media do cluster
df["dif_pct_cluster_geral"] = (df["preco"] - df["preco_medio_cluster_geral"]) / df["preco_medio_cluster_geral"]
df["dif_pct_cluster_bairro"] = (df["preco"] - df["preco_medio_cluster_bairro"]) / df["preco_medio_cluster_bairro"]

# Calcular o desvio padrão intra cluster geral e intra cluster bairro
df["std_cluster_geral"] = df.groupby("cluster_geral")["preco"].transform("std")
df["std_cluster_bairro"] = df.groupby(["bairro", "cluster_bairro"])["preco"].transform("std")

# Calcular o "Desconto" estatístico
df["z_value_cluster_geral"] = (df["preco"] - df["preco_medio_cluster_geral"]) / df["std_cluster_geral"] 
df["z_value_cluster_bairro"] = (df["preco"] - df["preco_medio_cluster_bairro"]) / df["std_cluster_bairro"] 

# Excluir timezone da colunad e data hotra
df["datahora"] = df["datahora"].apply(str)

# Separar apenas colunas necessárias
df = df[[
    "id", "datahora", "preco", "tamanho", "n_quartos", "n_banheiros",
    "n_garagem", "rua", "bairro", "cidade", "cluster_geral", 
    "cluster_bairro", "preco_medio_cluster_geral",
    "preco_medio_cluster_bairro", "dif_pct_cluster_geral",
    "dif_pct_cluster_bairro", "z_value_cluster_geral", 
    "z_value_cluster_bairro",
    "fonte_dado", 
    ]]

# Renomear colunas do entregável
df.columns = [
    "ID Imóvel", "DataHora", "Preço", "Tamanho (m²)", 
    "Qtd Quartos (#)", "Qtd Banheiros (#)", "Qtd Vagas Garagem (#)",
    "Rua", "Bairro", "Cidade", "Cluster Geral", "Cluster Bairro",
    "Preço médio do Imóvel (Cluster Geral)", "Preço médio do Imóvel (Cluster Bairro)",
    "Diferença % preço imóvel (Média Cluster Geral)",
    "Diferença % preço imóvel (Média Cluster Bairro)",
    "Z-Valor preço imóvel (Cluster Geral)",
    "Z-Valor preço imóvel (Cluster Bairro)",
    "Site Anúncio"
    ]

In [84]:
# Fazer dataframe de descrição dos clusteres
descritivo_clusters_bairro = df[["Bairro", "Cluster Bairro", "Preço", "Tamanho (m²)", "Qtd Quartos (#)", "Qtd Banheiros (#)", "Qtd Vagas Garagem (#)", "Z-Valor preço imóvel (Cluster Geral)", "Z-Valor preço imóvel (Cluster Bairro)",]]
descritivo_clusters = df[["Cluster Geral", "Preço", "Tamanho (m²)", "Qtd Quartos (#)", "Qtd Banheiros (#)", "Qtd Vagas Garagem (#)", "Z-Valor preço imóvel (Cluster Geral)", "Z-Valor preço imóvel (Cluster Bairro)",]]

# Passar .title no bairro
descritivo_clusters_bairro["Bairro"] = descritivo_clusters_bairro["Bairro"].str.title()

# Agrupar os dataframe por média
descritivo_clusters_bairro = descritivo_clusters_bairro.groupby(["Bairro", "Cluster Bairro"]).mean()
descritivo_clusters = descritivo_clusters.groupby("Cluster Geral").mean()

# Renomear colunas
descritivo_clusters_bairro = descritivo_clusters_bairro.rename(columns={c: "Média " + c for c in descritivo_clusters_bairro.columns})
descritivo_clusters = descritivo_clusters.rename(columns={c: "Média " + c for c in descritivo_clusters.columns})

# Arredondar casas numéricas
descritivo_clusters_bairro = descritivo_clusters_bairro.round(2)
descritivo_clusters = descritivo_clusters.round(2)

In [86]:
descritivo_clusters 

,Média Preço,Média Tamanho (m²),Média Qtd Quartos (#),Média Qtd Banheiros (#),Média Qtd Vagas Garagem (#),Média Z-Valor preço imóvel (Cluster Geral),Média Z-Valor preço imóvel (Cluster Bairro)
Cluster Geral,,,,,,,
0,791668.69,122.51,2.93,2.00,2.14,0.0,-0.12
1,1764721.44,262.66,3.24,4.01,4.23,0.0,0.66
2,714976.71,441.96,0.00,0.00,0.00,-0.0,-0.07
3,313640.69,42.59,0.59,1.00,0.30,-0.0,-0.39
4,885310.57,145.14,2.99,3.00,2.16,-0.0,-0.01
5,2515268.40,3285.20,0.00,0.00,0.00,0.0,0.70
6,2839635.68,404.71,4.02,6.40,4.33,0.0,1.26
7,1240245.83,278.42,3.25,2.79,6.92,0.0,-0.06
8,1183538.52,221.98,4.36,4.00,2.35,0.0,-0.03


In [55]:
# Fazer dataframe de descrição dos clusteres
descritivo_clusters_bairro = df[["Bairro", "Cluster Bairro", "Preço", "Tamanho (m²)", "Qtd Quartos (#)", "Qtd Banheiros (#)", "Qtd Vagas Garagem (#)"]]
descritivo_clusters = df[["Cluster Geral", "Preço", "Tamanho (m²)", "Qtd Quartos (#)", "Qtd Banheiros (#)", "Qtd Vagas Garagem (#)"]]

# Passar .title no bairro
descritivo_clusters_bairro["Bairro"] = descritivo_clusters_bairro["Bairro"].str.title()

# Agrupar os dataframe por média
descritivo_clusters_bairro = descritivo_clusters_bairro.groupby(["Bairro", "Cluster Bairro"]).mean()
descritivo_clusters = descritivo_clusters.groupby("Cluster Geral").mean()

# Renomear colunas
descritivo_clusters_bairro = descritivo_clusters_bairro.rename(columns={c: "Média " + c for c in descritivo_clusters_bairro.columns})
descritivo_clusters = descritivo_clusters.rename(columns={c: "Média " + c for c in descritivo_clusters.columns})

# Arredondar casas numéricas
descritivo_clusters_bairro = descritivo_clusters_bairro.round(2)
descritivo_clusters = descritivo_clusters.round(2)

# Ordenar por tamanho
descritivo_clusters_bairro = descritivo_clusters_bairro.sort_values(["Bairro", "Média Tamanho (m²)"])
descritivo_clusters = descritivo_clusters.sort_values("Média Tamanho (m²)")

# Definir estilo de borda médio
medium_border_style = Border(
    left=Side(style="medium"), 
    right=Side(style="medium"), 
    top=Side(style="medium"), 
    bottom=Side(style="medium")
)

# Definir estilo de borda thin
thin_border_style = Border(
    left=Side(style="dashed"), 
    right=Side(style="dashed"), 
    top=Side(style="dashed"), 
    bottom=Side(style="dashed")
)

# Fazer woorkbook
workbook = openpyxl.Workbook()

# Fazer sheets para armazenar os dados
worksheet_clusterizacao = workbook.create_sheet(f"Clusterizacao Imoveis")
worksheet_descritivo_cluster_bairro = workbook.create_sheet(f"Médias dos clusteres de bairro")
worksheet_descritivo_cluster_geral = workbook.create_sheet(f"Médias os clusteres gerais")

# Definir estilo do heading
default_heading_style = openpyxl.styles.NamedStyle(name="default_heading_style")
default_heading_style.font = Font(bold=True, color="FFFFFF")
default_heading_style.fill = PatternFill(start_color="F1F8A2", end_color="F1F8A2", fill_type="solid")

# Definir estilo das ceualr de valor
default_style = openpyxl.styles.NamedStyle(name="value_style")
default_style.fill = PatternFill(start_color="E7E7E7", end_color="E7E7E7", fill_type="solid")

# Fazer copia dos dados dos dataframes para evitar problemas de memória
worksheet_clusterizacao_data = df.copy()
worksheet_descritivo_cluster_bairro_data = descritivo_clusters_bairro.copy()
worksheet_descritivo_cluster_geral_data = descritivo_clusters.copy()

# Carregar as linhas do dataframe pandas
worksheet_clusterizacao_rows = dataframe_to_rows(worksheet_clusterizacao_data, index=False, header=True)
worksheet_descritivo_cluster_bairro_rows = dataframe_to_rows(worksheet_descritivo_cluster_bairro_data, index=False, header=True)
worksheet_descritivo_cluster_geral_rows = dataframe_to_rows(worksheet_descritivo_cluster_geral_data, index=False, header=True)

# Definir formato de cada coluna
openpyxl_formats = [
    'General', 'mm-dd-yy', 'R$   #,##0.00', '0', '0', 
    '0', '0', 'General', 'General', 'General',
    '0', '0',
    'R$   #,##0.00',
    'R$   #,##0.00',
    '0.00%',
    '0.00%',
    'General'
]


for rows, worksheet in [(worksheet_clusterizacao_rows, worksheet_clusterizacao), (worksheet_descritivo_cluster_bairro_rows, worksheet_descritivo_cluster_bairro), (worksheet_descritivo_cluster_geral_rows, worksheet_descritivo_cluster_geral)]:        
    # Iterar todas as linhas da worksheet atual e colocar os valores
    for r_idx, row in enumerate(rows, 1):
        for c_idx, value in enumerate(row, 1):
            worksheet.cell(row=r_idx, column=c_idx, value=value)

    # Mudar o estilo do heading
    for row in worksheet.iter_rows(min_row=0, max_row=1):
        # Em cada célula do geading
        for cell in row:
            # Mudar o estilo para estilo heading
            cell.style = default_heading_style

            # Mudar a borda
            cell.border = medium_border_style

    # Mudar o estilo das linhas
    for r_idx, row in enumerate(worksheet.iter_rows(min_row=2)):
        # Em cada célula
        for cell_idx, cell in enumerate(row):
            # Colcoar estilo default
            cell.style = default_style

            # Mudar a borda
            cell.border = thin_border_style

            # Mudar formatação
            cell.number_format = openpyxl_formats[cell_idx]

In [56]:
workbook.save("t.xlsx")

# Pre analysis

In [2]:
import duckdb
from unidecode import unidecode
from datetime import datetime
import logging
import os
import sys
import requests
import time
import joblib
import numpy as np
import json
import pandas as pd

from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

SELIC_RATE_ANNUAL = 14.50 / 100  # Annual SELIC rate (14.50%)

In [3]:
def impute_lat_lon(df_to_impute, df_knowledge_base, logger):
    """
    Imputes missing lat/lon in a dataframe using means from a knowledge base dataframe.
    If imputing a dataframe on itself, pass it for both arguments.
    """
    logger.info(f"Starting lat/lon imputation. Nulls before: Lat={df_to_impute['latitude'].isnull().sum()}, Lon={df_to_impute['longitude'].isnull().sum()}")
    
    df = df_to_impute.copy()

    # 1. Calculate means from the knowledge base
    city_mean_lat = df_knowledge_base['latitude'].mean()
    city_mean_lon = df_knowledge_base['longitude'].mean()

    if not (pd.notna(city_mean_lat) and pd.notna(city_mean_lon)):
        logger.warning("Could not calculate city-wide mean coordinates from knowledge base. Skipping imputation.")
        df.dropna(subset=['latitude', 'longitude'], inplace=True)
        return df

    logger.info(f"Using KB for imputation. City-wide mean: Lat={city_mean_lat:.4f}, Lon={city_mean_lon:.4f}")
    bairro_lat_map = df_knowledge_base.groupby('bairro')['latitude'].mean()
    bairro_lon_map = df_knowledge_base.groupby('bairro')['longitude'].mean()
    
    # 2. Map bairro-specific means for rows with missing lat/lon
    lat_na_mask = df['latitude'].isna()
    if lat_na_mask.any():
        df.loc[lat_na_mask, 'latitude'] = df.loc[lat_na_mask, 'bairro'].map(bairro_lat_map)

    lon_na_mask = df['longitude'].isna()
    if lon_na_mask.any():
        df.loc[lon_na_mask, 'longitude'] = df.loc[lon_na_mask, 'bairro'].map(bairro_lon_map)
    
    # 3. Fallback to city-wide mean for any remaining NaNs
    df['latitude'].fillna(city_mean_lat, inplace=True)
    df['longitude'].fillna(city_mean_lon, inplace=True)

    # 4. Final drop for safety
    df.dropna(subset=['latitude', 'longitude'], inplace=True)
    
    logger.info(f"Lat/lon imputation complete. Nulls after: Lat={df['latitude'].isnull().sum()}, Lon={df['longitude'].isnull().sum()}")

    return df

def calculate_adjusted_roi(purchase_price, estimated_resale_value, logger):
    """
    Calculates a more realistic ROI by subtracting fixed and variable costs,
    including opportunity cost based on the SELIC rate.

    Args:
        purchase_price (float): The price paid for the property.
        estimated_resale_value (float): The estimated value the property will be sold for.
        logger (logging.Logger): The logger instance.

    Returns:
        float: The adjusted ROI as a percentage, or 0.0 if costs exceed profit.
    """
    try:
        # 1. Transactional Costs on Purchase
        documentation_cost = purchase_price * 0.055
        auctioneer_commission = purchase_price * 0.05
        legal_fee = 5000  # Imissão na posse
        other_costs = 300
        
        total_purchase_costs = documentation_cost + auctioneer_commission + legal_fee + other_costs
        total_investment = purchase_price + total_purchase_costs

        # 2. Transactional Costs on Resale
        resale_broker_commission = estimated_resale_value * 0.06 # Using 6% as a standard broker fee

        # 3. Opportunity Cost (over 10 months)
        selic_10_months = ((1 + SELIC_RATE_ANNUAL)**(10/12)) - 1
        opportunity_cost = total_investment * selic_10_months

        # 4. Net Profit Calculation
        gross_profit = estimated_resale_value - total_investment
        net_profit = gross_profit - resale_broker_commission - opportunity_cost
        
        # 5. Adjusted ROI Calculation
        if total_investment <= 0:
            return 0.0
            
        adjusted_roi = (net_profit / total_investment) * 100
        
        return max(adjusted_roi, 0) # ROI can't be negative in this context; it's just a bad deal

    except Exception as e:
        logger.error(f"Error in calculate_adjusted_roi: {e}", exc_info=True)
        return 0.0

def setup_logging():
    """Set up logging configuration."""
    logger = logging.getLogger("run_pre_analysis")
    logger.setLevel(logging.INFO)
    
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)
    
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    
    return logger

logger = setup_logging()

def build_knn_model(df_kb, logger):
    """Builds an in-memory K-Nearest Neighbors model for finding comparable properties."""
    logger.info("Building K-Nearest Neighbors 'comps' model...")
    try:
        # Define the core features for finding 'comparable' properties
        knn_features = ['tamanho', 'n_quartos', 'n_banheiros', 'n_garagem', 'latitude', 'longitude']
        
        # Ensure the KB has the required data and is clean
        df_kb_knn = df_kb.dropna(subset=knn_features + ['preco']).copy()

        if df_kb_knn.empty:
            logger.warning("Not enough data in knowledge base to build KNN model.")
            return None, None

        X_knn = df_kb_knn[knn_features]
        y_knn = df_kb_knn['preco']
        
        # A scaler is crucial for distance-based algorithms like KNN
        scaler = StandardScaler()
        X_knn_scaled = scaler.fit_transform(X_knn)
        
        # Initialize and "fit" the KNN model (it just stores the data)
        knn_model = KNeighborsRegressor(n_neighbors=7, n_jobs=-1)
        knn_model.fit(X_knn_scaled, y_knn)
        
        logger.info(f"KNN 'comps' model built successfully with {len(df_kb_knn)} properties.")
        return knn_model, scaler
    except Exception as e:
        logger.error(f"Failed to build KNN model: {e}", exc_info=True)
        return None, None

def impute_leilao_data(df, imputers, logger):
    """Imputes missing values in auction data using pre-trained imputers."""
    df_copy = df.copy()
    logger.info("Imputing missing values in auction data...")

    # Impute n_quartos and n_banheiros using the loaded Linear Regression models
    for col in ['n_quartos', 'n_banheiros']:
        if col not in df_copy.columns:
            logger.warning(f"Column '{col}' not found in auction data. Creating it for imputation.")
            df_copy[col] = np.nan
            
        imputer_model = imputers.get(f'{col}_imputer')
        if imputer_model:
            # Predict only for rows with missing values in the target column but valid 'tamanho'
            missing_mask = df_copy[col].isna() & df_copy['tamanho'].notna()
            if missing_mask.any():
                # The imputer model was trained on 'tamanho'
                features_for_imputation = df_copy.loc[missing_mask, ['tamanho']]
                predicted_values = imputer_model.predict(features_for_imputation)
                df_copy.loc[missing_mask, col] = np.round(predicted_values).clip(1)

    # Impute n_garagem using the saved median value
    if 'n_garagem' not in df_copy.columns:
        df_copy['n_garagem'] = np.nan
    garagem_imputer = imputers.get('n_garagem_imputer')
    if garagem_imputer is not None:
        df_copy['n_garagem'].fillna(garagem_imputer, inplace=True)

    # Fill any remaining NaNs after imputation (e.g., if area_util was NaN) with the median
    for col in ['n_quartos', 'n_banheiros', 'n_garagem']:
        if col in df_copy.columns and df_copy[col].isna().any():
            fallback_median = imputers.get('n_garagem_imputer', 1) # Use garage median or 1 as a fallback
            df_copy[col].fillna(fallback_median, inplace=True)
        # Ensure integer types
        df_copy[col] = df_copy[col].astype(int)
        
    logger.info("Auction data imputation complete.")
    return df_copy

def feature_engineer_leilao_data(df, kmeans_model, logger):
    """Applies feature engineering to auction data."""
    df_copy = df.copy()
    logger.info("Applying feature engineering to auction data...")

    # A. Geospatial Clustering (using the loaded K-Means model)
    if 'latitude' in df_copy.columns and 'longitude' in df_copy.columns:
        # Ensure no NaN values in coordinates before predicting
        coord_mask = df_copy[['latitude', 'longitude']].notna().all(axis=1)
        if coord_mask.any():
            df_copy.loc[coord_mask, 'bairro_cluster'] = kmeans_model.predict(df_copy.loc[coord_mask, ['latitude', 'longitude']])
            df_copy['bairro_cluster'] = df_copy['bairro_cluster'].astype('category')
            logger.info("Geospatial clusters assigned.")
    else:
        logger.warning("Latitude/Longitude not available, skipping K-Means clustering.")

    # B. Distance to Points of Interest
    logger.info("Calculating distances to points of interest for auction data...")
    poi_path = os.path.join(_PROJECT_ROOT, "data", "model_training", "points_of_interest.json")
    try:
        with open(poi_path, 'r', encoding='utf-8') as f:
            points_of_interest = json.load(f)
        
        valid_pois = [p for p in points_of_interest if p.get('latitude') is not None and p.get('longitude') is not None]
        logger.info(f"Loaded {len(valid_pois)} valid POIs for distance calculation.")

        if 'latitude' in df_copy.columns and 'longitude' in df_copy.columns:
            for poi in valid_pois:
                point_name = poi['point_name']
                sanitized_name = ''.join(e for e in point_name if e.isalnum() or e.isspace()).replace(' ', '_').lower()
                feature_name = f"dist_{sanitized_name}"
                
                poi_lat, poi_lon = poi['latitude'], poi['longitude']
                
                df_copy[feature_name] = np.sqrt(
                    (df_copy['latitude'] - poi_lat)**2 + (df_copy['longitude'] - poi_lon)**2
                )
            logger.info(f"Created {len(valid_pois)} distance features for auction data.")
        else:
            logger.warning("Latitude/Longitude not available in auction data, skipping POI distance calculation.")

    except Exception as e:
        logger.error(f"Could not calculate POI distances for auction data: {e}. Some features will be missing.")

    # C. Interaction and Ratio Features
    if 'tamanho' in df_copy.columns and 'n_quartos' in df_copy.columns and 'n_banheiros' in df_copy.columns:
        epsilon = 1e-6
        df_copy['tamanho_por_quarto'] = df_copy['tamanho'] / (df_copy['n_quartos'] + epsilon)
        df_copy['banheiros_por_quarto'] = df_copy['n_banheiros'] / (df_copy['n_quartos'] + epsilon)
        logger.info("Interaction features created.")
    else:
        logger.warning("Missing columns for interaction features, skipping.")

    return df_copy

def kb_data_cleaning(df):
    """
    Clean and preprocess the knowledge base dataframe.
    """
    df = df.drop_duplicates(subset=["id"])
    df = df.loc[(df["n_quartos"] > 0) & (df["n_quartos"] < 10)]
    df = df.loc[(df["n_banheiros"] > 0) & (df["n_banheiros"] < 10)]
    df = df.loc[(df["tamanho"] > 10) & (df["tamanho"] < 1000)]
    df = df.loc[(df["preco"] > 10_000) & (df["preco"] < 10_000_000)]
    df = df.loc[df["bairro"].notna()]
    df["bairro"] = df["bairro"].apply(lambda x: unidecode(x.lower().strip()))
    df["preco_m2"] = df["preco"] / df["tamanho"]
    df = df.loc[df["preco_m2"] < 30_000]
    return df

def leilao_data_cleaning(df):
    """
    Clean and preprocess the auction dataframe based on the new schema.
    """
    df_leilao = df.copy()
    df_leilao = df_leilao.applymap(lambda x: None if x == "<NA>" else x)
    df_leilao = df_leilao.drop_duplicates(subset=["id"])

    # Rename columns to match the historical schema used across the script
    df_leilao = df_leilao.rename(columns={
        'preco_primeira_praca': 'preco',
        'area_util': 'tamanho',
        'link_detalhes': 'url'
    })

    # Ensure essential numeric columns are correctly typed, coercing errors to NaN
    for col in ['preco', 'tamanho', 'preco_atual']:
        if col in df_leilao.columns:
            df_leilao[col] = pd.to_numeric(df_leilao[col], errors='coerce')
    
    # Drop rows that are unusable for analysis due to missing core information
    df_leilao.dropna(subset=['preco', 'tamanho', 'bairro', 'preco_atual'], inplace=True)

    # Calculate price per square meter
    df_leilao['preco_m2'] = df_leilao['preco'] / df_leilao['tamanho']

    # Standardize neighborhood names
    df_leilao["bairro"] = df_leilao["bairro"].apply(lambda x: unidecode(str(x).lower().strip()))
    return df_leilao

In [5]:
con = duckdb.connect(database=r"C:\Users\PedroMiyasaki\OneDrive - DHAUZ\Área de Trabalho\Projetos\PESSOAL\kodomiya\kodomiya\db\kodomiya.duckdb")

# Select all columns needed for knowledge base
kb_querys = [
    "SELECT preco, tamanho, n_quartos, n_banheiros, n_garagem, bairro, latitude, longitude, id FROM kodomiya_chaves_na_mao.chaves_na_mao_register",
    "SELECT preco, tamanho, n_quartos, n_banheiros, n_garagem, bairro, latitude, longitude, id FROM kodomiya_viva_real.viva_real_register",
    "SELECT preco, tamanho, n_quartos, n_banheiros, n_garagem, bairro, latitude, longitude, id FROM kodomiya_zap_imoveis.zap_imoveis_register"
]

# Fetch knowledge base data
df_kb_list = [con.execute(query).fetch_df() for query in kb_querys]

# Concatenate knowledge base data
df_kb = pd.concat(df_kb_list, ignore_index=True)

# Fetch leilao data
today = datetime.now().strftime("%Y-%m-%d")
df_leilao = con.execute(f"SELECT * FROM kodomiya_leilao_imovel.leilao_imovel_register WHERE data_primeira_praca	< '{today}' OR data_segunda_praca < '{today}'").fetchdf()

logger.info(f"Loaded {len(df_kb)} properties from knowledge base and {len(df_leilao)} open auctions.")

# Data Cleaning
df_kb = kb_data_cleaning(df_kb)
df_leilao = leilao_data_cleaning(df_leilao)

# Impute Lat/Lon for both dataframes using the knowledge base as the source of truth
logger.info("--- Imputing coordinates for Knowledge Base ---")
df_kb = impute_lat_lon(df_kb, df_kb, logger) # Impute KB on itself
logger.info("--- Imputing coordinates for Auction Data ---")
df_leilao = impute_lat_lon(df_leilao, df_kb, logger) # Impute auction data using KB stats

2025-06-09 18:59:30,833 - run_pre_analysis - INFO - Loaded 5248 properties from knowledge base and 2 open auctions.
2025-06-09 18:59:30,860 - run_pre_analysis - INFO - --- Imputing coordinates for Knowledge Base ---
2025-06-09 18:59:30,864 - run_pre_analysis - INFO - Starting lat/lon imputation. Nulls before: Lat=107, Lon=107
2025-06-09 18:59:30,867 - run_pre_analysis - INFO - Using KB for imputation. City-wide mean: Lat=-25.4510, Lon=-49.2720
2025-06-09 18:59:30,881 - run_pre_analysis - INFO - Lat/lon imputation complete. Nulls after: Lat=0, Lon=0
2025-06-09 18:59:30,883 - run_pre_analysis - INFO - --- Imputing coordinates for Auction Data ---
2025-06-09 18:59:30,887 - run_pre_analysis - INFO - Starting lat/lon imputation. Nulls before: Lat=1, Lon=1
2025-06-09 18:59:30,890 - run_pre_analysis - INFO - Using KB for imputation. City-wide mean: Lat=-25.4514, Lon=-49.2722
2025-06-09 18:59:30,899 - run_pre_analysis - INFO - Lat/lon imputation complete. Nulls after: Lat=0, Lon=0


C:\Users\PedroMiyasaki\AppData\Local\Temp\ipykernel_4944\1713580228.py:250: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_leilao = df_leilao.applymap(lambda x: None if x == "<NA>" else x)
C:\Users\PedroMiyasaki\AppData\Local\Temp\ipykernel_4944\1713580228.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['latitude'].fillna(city_mean_lat, inplace=True)
C:\Users\PedroMiyasaki\AppData\Local\Temp\ipykernel_4944\1713580228.py:34: FutureWarning: A value is trying to be set on a copy of 

In [6]:
df_leilao_zscore = df_leilao.copy()

neighborhood_stats = df_kb.groupby('bairro').agg(
    mean_preco_m2=('preco_m2', 'mean'),
    std_preco_m2=('preco_m2', 'std')
).reset_index()

df_leilao_zscore = df_leilao_zscore.merge(neighborhood_stats, on='bairro', how='left')

df_leilao_zscore['z_score'] = (
    (df_leilao_zscore['preco_m2'] - df_leilao_zscore['mean_preco_m2']) / 
        df_leilao_zscore['std_preco_m2']
)

undervalued_properties = df_leilao_zscore[df_leilao_zscore['z_score'] < -1.5].copy()
undervalued_properties.loc[:, 'z_score_rank'] = undervalued_properties['z_score'].rank(ascending=True)
undervalued_properties.loc[:, 'estimated_market_value_zscore'] = undervalued_properties['mean_preco_m2'] * undervalued_properties['tamanho']

undervalued_properties['z_score_roi'] = undervalued_properties.apply(
    lambda row: calculate_adjusted_roi(row['preco_atual'], row['estimated_market_value_zscore'], logger),
    axis=1
)

In [ ]:


# Build KNN 'comps' model from knowledge base
knn_model, knn_scaler = build_knn_model(df_kb, logger)

# A. Z-Score Based Analysis (Undervalued Properties)
# ----------------------------------------------------------------
logger.info("Starting Z-Score based analysis...")
df_leilao_zscore = df_leilao.copy()

neighborhood_stats = df_kb.groupby('bairro').agg(
    mean_preco_m2=('preco_m2', 'mean'),
    std_preco_m2=('preco_m2', 'std')
).reset_index()

df_leilao_zscore = df_leilao_zscore.merge(neighborhood_stats, on='bairro', how='left')

df_leilao_zscore['z_score'] = (
    (df_leilao_zscore['preco_m2'] - df_leilao_zscore['mean_preco_m2']) / 
        df_leilao_zscore['std_preco_m2']
)

undervalued_properties = df_leilao_zscore[df_leilao_zscore['z_score'] < -1.5].copy()
undervalued_properties.loc[:, 'z_score_rank'] = undervalued_properties['z_score'].rank(ascending=True)
undervalued_properties.loc[:, 'estimated_market_value_zscore'] = undervalued_properties['mean_preco_m2'] * undervalued_properties['tamanho']

undervalued_properties['z_score_roi'] = undervalued_properties.apply(
    lambda row: calculate_adjusted_roi(row['preco_atual'], row['estimated_market_value_zscore'], logger),
    axis=1
)

logger.info(f"Found {len(undervalued_properties)} potentially undervalued properties (Z-score).")

all_opportunities = undervalued_properties[['id', 'preco_atual', 'bairro', 'tamanho', 'url', 'rua', 'z_score', 'z_score_rank', 'z_score_roi']].copy()

# B. Regression Model Analysis
# ----------------------------------------------------------------
if model is not None and imputers is not None and kmeans_model is not None:
    logger.info("Starting regression model-based analysis...")
    df_leilao_model = df_leilao.copy()
    
    df_leilao_model = impute_leilao_data(df_leilao_model, imputers, logger)
    df_leilao_model = feature_engineer_leilao_data(df_leilao_model, kmeans_model, logger)

    model_features = model.feature_names_in_            
    df_leilao_model['predicted_price'] = model.predict(df_leilao_model[model_features])

    df_leilao_model['model_roi'] = df_leilao_model.apply(
        lambda row: calculate_adjusted_roi(row['preco_atual'], row['predicted_price'], logger),
        axis=1
    )

    df_leilao_model = df_leilao_model[df_leilao_model['model_roi'] > 20].copy()
    df_leilao_model.loc[:, 'regression_model_rank'] = df_leilao_model['model_roi'].rank(ascending=False)

    logger.info(f"Found {len(df_leilao_model)} potentially undervalued properties (Regression Model).")
    
    model_results = df_leilao_model[['id', 'model_roi', 'regression_model_rank']].copy()
    all_opportunities = all_opportunities.merge(model_results, on='id', how='outer')
else:
    logger.warning("Skipping regression analysis as model artifacts are not loaded.")

# C. K-Nearest Neighbors ('Comps') Analysis
# ----------------------------------------------------------------
if knn_model is not None and knn_scaler is not None and imputers is not None:
    logger.info("Starting K-Nearest Neighbors 'comps' analysis...")
    df_leilao_knn = df_leilao.copy()
    
    knn_features = ['tamanho', 'n_quartos', 'n_banheiros', 'n_garagem', 'latitude', 'longitude']
    
    df_leilao_knn_imputed = impute_leilao_data(df_leilao_knn, imputers, logger)

        
    X_leilao_knn = df_leilao_knn_imputed[knn_features]
    X_leilao_knn_scaled = knn_scaler.transform(X_leilao_knn)

    df_leilao_knn_imputed['knn_predicted_price'] = knn_model.predict(X_leilao_knn_scaled)
    
    df_leilao_knn_imputed['knn_roi'] = df_leilao_knn_imputed.apply(
        lambda row: calculate_adjusted_roi(row['preco_atual'], row['knn_predicted_price'], logger),
        axis=1
    )
    
    df_leilao_knn_imputed = df_leilao_knn_imputed[df_leilao_knn_imputed['knn_roi'] > 20].copy()
    df_leilao_knn_imputed.loc[:, 'knn_rank'] = df_leilao_knn_imputed['knn_roi'].rank(ascending=False)
    
    logger.info(f"Found {len(df_leilao_knn_imputed)} potentially undervalued properties (KNN 'Comps').")

    knn_results = df_leilao_knn_imputed[['id', 'knn_roi', 'knn_rank']].copy()
    all_opportunities = all_opportunities.merge(knn_results, on='id', how='outer')
else:
    logger.warning("Skipping KNN 'comps' analysis as model is not built.")
    
# D. Final Score Calculation & Ranking
# ----------------------------------------------------------------
if all_opportunities.empty:
    logger.info("No potentially undervalued properties found by any method.")
    return
    
# Re-populate descriptive data for all opportunities to fix NaNs from outer merges
analysis_cols = ['id', 'z_score', 'z_score_rank', 'z_score_roi', 'model_roi', 'regression_model_rank', 'knn_roi', 'knn_rank']
detail_cols = ['preco_atual', 'bairro', 'tamanho', 'url', 'rua', 'descricao']

analysis_cols_present = [c for c in analysis_cols if c in all_opportunities.columns]
detail_cols_present = ['id'] + [c for c in detail_cols if c in df_leilao.columns]

all_opportunities = all_opportunities[analysis_cols_present].merge(df_leilao[detail_cols_present], on='id', how='left')
    
all_opportunities['z_score_rank'].fillna(999, inplace=True)
all_opportunities['regression_model_rank'].fillna(999, inplace=True)
all_opportunities['knn_rank'].fillna(999, inplace=True)
all_opportunities.fillna({'z_score_roi': 0, 'model_roi': 0, 'knn_roi': 0}, inplace=True)

all_opportunities['final_score'] = (
    all_opportunities['z_score_rank'] * 0.2 + 
    all_opportunities['regression_model_rank'] * 0.5 +
    all_opportunities['knn_rank'] * 0.3
)

final_opportunities = all_opportunities[
    (all_opportunities['z_score_rank'] != 999) | 
    (all_opportunities['regression_model_rank'] != 999) |
    (all_opportunities['knn_rank'] != 999)
].sort_values(by='final_score', ascending=True)

logger.info(f"Generated a final ranked list of {len(final_opportunities)} properties.")
if final_opportunities.empty:
    logger.info("No promising opportunities found after final analysis. Exiting.")
    return

# E. Send Telegram Notifications
# ----------------------------------------------------------------
logger.info("Sending Telegram notifications for top properties...")
for _, row in final_opportunities.head(10).iterrows():
    try:
        z_score_rank_text = str(int(row['z_score_rank'])) if row['z_score_rank'] != 999 else "N/A"
        model_rank_text = str(int(row['regression_model_rank'])) if row['regression_model_rank'] != 999 else "N/A"
        knn_rank_text = str(int(row['knn_rank'])) if row['knn_rank'] != 999 else "N/A"

        zscore_roi_text, model_roi_text, knn_roi_text = "", "", ""
        mean_roi_values = []
        
        if row['z_score_roi'] > 0:
            zscore_roi_text = f"*- ROI (Z-Score):* {row['z_score_roi']:.2f}%\n"
            mean_roi_values.append(row['z_score_roi'])
            
        if row['model_roi'] > 0:
            model_roi_text = f"*- ROI (Modelo ML):* {row['model_roi']:.2f}%\n"
            mean_roi_values.append(row['model_roi'])
            
        if row['knn_roi'] > 0:
            knn_roi_text = f"*- ROI (KNN Comps):* {row['knn_roi']:.2f}%\n"
            mean_roi_values.append(row['knn_roi'])

        mean_roi = np.mean(mean_roi_values) if mean_roi_values else 0.0

        message = (
            f"*{escape_markdown('🚨 NOVA OPORTUNIDADE DE LEILÃO DETECTADA 🚨')}*\n\n"
            f"*{escape_markdown('Score Final:')}* `{row['final_score']:.2f}`\n"
            f"*{escape_markdown('ROI Médio (Ajustado):')}* `{mean_roi:.2f}%`\n\n"
            f"--- DETALHES ---\n"
            f"*{escape_markdown('ID do Imóvel:')}* `{escape_markdown(str(row['id']))}`\n"
            f"*{escape_markdown('Rua:')}* {escape_markdown(str(row['rua']))}\n"
            f"*{escape_markdown('Bairro:')}* {escape_markdown(row['bairro'])}\n"
            f"*{escape_markdown('Tamanho:')}* {row['tamanho']:.0f} m²\n"
            f"*{escape_markdown('Lance Inicial:')}* R$ {row['preco_atual']:,.2f}\n\n"
            f"--- ANÁLISE ---\n"
            f"{zscore_roi_text}"
            f"{model_roi_text}"
            f"{knn_roi_text}\n"
            f"*{escape_markdown('Link:')}* [Clique aqui para ver]({row['url']})\n\n"
            f"_{escape_markdown('Ranking Z-Score:')}_ `{z_score_rank_text}`\n"
            f"_{escape_markdown('Ranking Modelo ML:')}_ `{model_rank_text}`\n"
            f"_{escape_markdown('Ranking KNN Comps:')}_ `{knn_rank_text}`\n"
        )
        
        send_telegram_message(message)
        time.sleep(2)
    except Exception as e:
        logger.error(f"Failed to generate/send message for property {row.get('id', 'N/A')}: {e}", exc_info=True)
        continue

In [2]:
f"*- ROI (Z-Score):* {0.0:.2f}%\n"

'*- ROI (Z-Score):* 0.00%\n'